# USGS Organizational Setting

USGS Mission Areas are actually a somewhat elusive organizational construct despite their importance as the topmost organizing element in the USGS. They do have budgetary codes that fund the office function of the Mission Area, but they most often show up as a connecting reference via acronyms or other codes. To accommodate the connection, we introduce simple entities through a specialized API that provides basic nodes and relationships to the Associate Director in each case. Regions are similarly elusive and somewhat in flux over the last number of years with not all information systems and references fully meshing with the "Interior Unified Regions."

In [1]:
import isaid_helpers
import pandas as pd

In [2]:
pd.read_csv(isaid_helpers.f_mas_n_regions).head()

,_source_name,_source_reference,_source_source_id,director_email,director_name,director_title,name,sipp_code,url
0,USGS Mission Areas,https://www.usgs.gov/science/mission-areas,mission_areas,kgallagher@usgs.gov,Kevin T Gallagher,"Associate Director, Core Science Systems",Core Science Systems Mission Area,CSS,https://www.usgs.gov/mission-areas/core-scienc...
1,USGS Mission Areas,https://www.usgs.gov/science/mission-areas,mission_areas,akinsinger@usgs.gov,Anne Kinsinger,"Associate Director, Ecosystems",Ecosystems Mission Area,EMA,https://www.usgs.gov/mission-areas/ecosystems
2,USGS Mission Areas,https://www.usgs.gov/science/mission-areas,mission_areas,sryker@usgs.gov,"Sarah Ryker, Ph.D.","Associate Director, Energy and Minerals",Energy and Minerals Mission Area,EM,https://www.usgs.gov/mission-areas/energy-and-...
3,USGS Mission Areas,https://www.usgs.gov/science/mission-areas,mission_areas,dcline@usgs.gov,"Don Cline, Ph.D.","Associate Director, Water Resources",Water Resources Mission Area,WMA,https://www.usgs.gov/mission-areas/water-resou...
4,USGS Mission Areas,https://www.usgs.gov/science/mission-areas,mission_areas,applegate@usgs.gov,"Dave Applegate, Ph.D.","Associate Director, Natural Hazards",Natural Hazards Mission Area,NH,https://www.usgs.gov/mission-areas/natural-haz...


In [3]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
    LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
    WITH row
        MERGE (o:Organization {name: row.name}) 
        ON CREATE
            SET o.url = row.url,
            o.category = row._source_name,
            o.reference = row._source_reference,
            o.sipp_code = row.sipp_code

        WITH o, row
            MERGE (d:Person {email: row.director_email}) 
            ON CREATE
                SET d.name = row.director_name,
                d.original_source = row._source_reference,
                d.title = row.director_title
            MERGE (d)-[:DIRECTOR_OF]->(o)
            
        WITH d, row
            MERGE (t:JobTitle {name: row.director_title})

            MERGE (d)-[rt:JOB_TITLE]->(t)
                SET rt.reference = row._source_reference
    """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_mas_n_regions
    })

CPU times: user 1.07 ms, sys: 1.47 ms, total: 2.55 ms
Wall time: 108 ms


# USGS Cost Centers

Organizational units in USGS are challenging overall with no one master source and a number of different interpretations of the different types of units we have. One source that we do incorporate here comes from the idea of "Cost Center," which is a budgetary term indicating where allocated funding flows in the organization. We use a slimmed down version of basic Cost Center information that is derived from an internal system and stripped down to its basics for graphing purposes. It includes a number of different kinds of "centers," some of which are "Science Centers" and others that are "Programs," along with relationships to "Regions" and "Mission Areas" and sub-organizations within larger Cost Centers. This does give us useful identifiers that we include as "fbms_code" properties that link people to organizational units from other sources.

In [4]:
pd.read_csv(isaid_helpers.f_usgs_centers).head()

,CenterCode,CenterName,RegionCode,WSC,MissionArea,SubBureauCode,Active,CenterDirectorName,CenterDirectorEmail,CostCenterPrimary,MissionAreaName,RegionName,parent_cost_center_code,LastChangeInBASIS
0,RA,Office of the Rocky Mountain Regional Director,RM,N,REG,CM,Y,Peter Griffiths,pggriffi@usgs.gov,GGCMRA0000,NaN,NaN,NaN,NaN
1,RB,Fort Collins Science Center,RM,N,EMA,CM,Y,Sharon Taylor,sktaylor@usgs.gov,GGCMRB0000,Ecosystems Mission Area,NaN,NaN,NaN
2,RB,SW ECOSYSTEMS FLD STA-ALBUQUERQUE,RM,N,EMA,CM,Y,Sharon Taylor,sktaylor@usgs.gov,GGCMRB0100,Ecosystems Mission Area,NaN,GGCMRB0000,2019-11-05
3,RB,SW ECOSYSTEMS FLD STA-JEMEZ MTNS.,RM,N,EMA,CM,Y,Sharon Taylor,sktaylor@usgs.gov,GGCMRB0200,Ecosystems Mission Area,NaN,GGCMRB0000,2019-11-05
4,RE,Colorado Water Science Center,RM,Y,WMA,CM,Y,David Mau,dpmau@usgs.gov,GGCMRE0000,Water Resources Mission Area,NaN,NaN,NaN


In [5]:
%%time
with isaid_helpers.graph_driver.session(database=isaid_helpers.graphdb) as session:
    session.run("""
        LOAD CSV WITH HEADERS FROM '%(source_path)s/%(source_file)s' AS row
        WITH row
            MERGE (o:Organization {name: row.CenterName}) 
            ON CREATE
                SET o.sipp_CenterCode = row.CenterCode,
                o.original_source = "SIPP Centers",
                o.fbms_code = row.CostCenterPrimary,
                o.sipp_SubBureauCode = row.SubBureauCode,
                o.category = "USGS Cost Center"

            ON MATCH
                SET o.sipp_CenterCode = row.CenterCode,
                o.fbms_code = row.CostCenterPrimary,
                o.sipp_SubBureauCode = row.SubBureauCode,
                o.category = "USGS Cost Center"

            WITH o, row
                WHERE NOT row.MissionAreaName IS NULL
                    MERGE (ma:Organization {name: row.MissionAreaName})
                    MERGE (o)-[:PART_OF]->(ma)

            WITH o, row
                WHERE NOT row.RegionName IS NULL
                    MERGE (r:Organization {name: row.RegionName})
                    MERGE (o)-[:PART_OF]->(r)

            WITH o, row
                WHERE NOT row.parent_cost_center_code IS NULL
                    MERGE (po:Organization {fbms_code: row.parent_cost_center_code})
                    MERGE (o)-[:PART_OF]->(po)

            WITH o, row
                WHERE NOT row.CenterDirectorEmail IS NULL
                    MERGE (p:Person {email: row.CenterDirectorEmail})
                    ON CREATE
                        SET p.name = row.CenterDirectorName
                    MERGE (p)-[:DIRECTOR_OF]->(o)
        """ % {
        "source_path": isaid_helpers.local_cache_path,
        "source_file": isaid_helpers.f_usgs_centers
    })

CPU times: user 1.36 ms, sys: 1.44 ms, total: 2.8 ms
Wall time: 239 ms
